In [1]:
import pandas as pd
import numpy as np
import ephem
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [2]:
csvFiles = [f for f in os.listdir('texas_county')]
    

In [3]:
csvFiles[:10]

['Callahan County.csv',
 'Childress County.csv',
 'Fort Bend County.csv',
 'Brooks County.csv',
 'Tarrant County.csv',
 'Potter County.csv',
 'Marion County.csv',
 'Jack County.csv',
 'Sabine County.csv',
 'Howard County.csv']

In [4]:
def EndtimeCivilTwilight(dates):
    
    '''
    End civil twilight function takes in the dates for all the stops in the 
    paper. 
    
    '''
    
    # set the location Houston
    end_twilight = []

    # set the location Houston
    obs = ephem.Observer()
    
    # converting the lat lon to radian which is the unit for ephem
    obs.lat = np.deg2rad(mean_lat)
    obs.long = np.deg2rad(mean_lon)

    obs.horizon = '-6'

    # all the civil twilight time for each unique time on which the stop occurred
    for d in all_dates_str:
        obs.date = d

        #foo = ephem.Date(houston.next_setting(ephem.Sun(), use_center=True))
        foo = obs.next_setting(ephem.Sun(), use_center=True)

        end_twilight.append(ephem.localtime(foo))

    return end_twilight

In [5]:
def beforeAfter(df, end_twilight):
    
    '''
    This function suppose to tell you 
    It takes input of the dataframe and a list of end civil twilight time for each date
    the output is a string for each stop determines whether stop occurs before or after 
    the sun has set. For before darkness fallen is marked BD and after darkness is marked
    with AD
    '''
    
# calculate the before dailight or after day light for each stops 
    bf_af = []

    for i in range(df.shape[0]):

        # get the timestamps for stops
        ts = df.iloc[i, 27]

        # check if the date will twilight 
        for d in range(len(end_twilight)):
            
            # find for each date
            if (ts.date() == end_twilight[d].date()):
        
                # compare the timestamp to the end civil twilight time and mark with in
                if (ts < end_twilight[d]):
                    bf_af.append('BD')

                else:      
                    bf_af.append('AD')
                    
    return bf_af


In [6]:
def ratioFactor(df):

    wb = df[df.driver_race == 'White'].groupby('light_or_dark').count().iloc[0, 0]
    wa = df[df.driver_race == 'White'].groupby('light_or_dark').count().iloc[1, 0]
    
    bb = df[df.driver_race == 'Black'].groupby('light_or_dark').count().iloc[0, 0]
    ba = df[df.driver_race == 'Black'].groupby('light_or_dark').count().iloc[1, 0]
    
    hb = df[df.driver_race == 'Hispanic'].groupby('light_or_dark').count().iloc[0, 0]
    ha = df[df.driver_race == 'Hispanic'].groupby('light_or_dark').count().iloc[1, 0]
    
    # black
    br = ((bb / (wb + bb + hb)), (ba / (wa + ba + ha)))
    wr = ((wb / (wb + bb + hb)), (wa / (wa + ba + ha)))
    hr = ((hb / (wb + bb + hb)), (ha / (wa + ba + ha)))
    
    return br, wr, hr

In [13]:
# iterating through the file by their county files
l = []

for f in csvFiles[:10]:
    
    df = pd.read_csv('texas_county' + '/' + f)
    
    #df = df[df['lat'].notnull() & df['lon'].notnull()]
    
    # calculating the average lat and lon
    mean_lat = df.lat.mean()
    mean_lon = df.lon.mean()
    
    # Find out the unqie dates for the 
    all_dates_str = df['stop_date'].unique()  + [' 00:00:00'] * len(df['stop_date'].unique())
    
    #create datetime objects for all the stop time
    df['stop_date_time'] = pd.to_datetime(df['stop_date'] + ' ' + df['stop_time'])
    
    # calculating the end of civil twilight for each date available 
    end_twilight = EndtimeCivilTwilight(all_dates_str)
    
    # Produce a 
    bf_af = beforeAfter(df, end_twilight)
    
    # attach the before or after 
    df['light_or_dark'] = pd.Series(bf_af)
    
    wr, br, hr = ratioFactor(df)
    
    l.append((wr, br, hr))

/Users/tedu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (17,20,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/tedu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
l

[((0.1053144807411019, 0.10849924258818437),
  (0.73362587355761422, 0.7191084180913222),
  (0.16105964570128392, 0.17239233932049339)),
 ((0.071593533487297925, 0.067099020894317143),
  (0.77598152424942268, 0.76849702230745942),
  (0.15242494226327943, 0.16440395679822348)),
 ((0.24005230290596749, 0.23450252798844348),
  (0.46351195978086884, 0.47062116287468403),
  (0.29643573731316364, 0.29487630913687252)),
 ((0.017570365396452355, 0.017184542685601836),
  (0.24385876559789604, 0.25217783682831751),
  (0.73857086900565161, 0.73063762048608061)),
 ((0.15562868504044974, 0.15645889037433156),
  (0.65953654188948307, 0.66972342914438499),
  (0.18483477307006718, 0.17381768048128343)),
 ((0.069281045751633991, 0.068406921914130364),
  (0.72438940488476089, 0.7188705638156262),
  (0.20632954936360509, 0.21272251427024344)),
 ((0.18826858200021765, 0.18684659090909092),
  (0.74589182718467728, 0.75207386363636364),
  (0.065839590815105015, 0.061079545454545456)),
 ((0.03768175302068400